LD Docs https://www.cog-genomics.org/plink/1.9/ld

In [1]:
import scipy
print(scipy.__version__)
import sgkit as sg

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [6]:
import pandas as pd

pd.read_csv("data/genotypes_matrix.txt", compression='gzip')

\t00077E7A10\t00077E7AB7\t00077E9536\t00077E9B37\t00077EA639\t00077EAA6C\t00077EAF65\t00078A1795\t933000120117306\t933000120117307\t933000120117311\t933000120117312\t933000120117313\t933000120117318\t933000120117320\t933000120117321\t933000120117323\t933000120117324\t933000120117325\t933000120117326\t933000120117329\t933000120117331\t933000120117332\t933000120117333\t933000120117334\t933000120117336\t933000120117341\t933000120117342\t933000120117344\t933000120117346\t933000120117347\t933000120117348\t933000120117349\t933000120117350\t933000120117351\t933000120124696\t933000120124697\t933000120124698\t933000120124699\t933000120124704\t933000120124705\t933000120138279\t933000120138280\t933000120138282\t933000120138283\t933000120138285\t933000120138286\t933000120138287\t933000120138288\t933000120138293\t933000120138295\t933000120138300\t933000120138302\t933000120138304\t933000120138305\t933000120138306\t933000120138316\t933000120138321\t933000120138322\t933000120138324\t933000120138330\t933000120138331\t933000120138332\t933000120138334\t933000120138336\t933000120138338\t933000120138339\t933000120138340\t933000120138341\t933000120138342\t933000120138343\t933000120138344\t933000120138346\t933000120138347\t933000120138348\t933000120138349\t933000120138351\t933000120138354\t933000120138385\t933000120138386\t933000120138388\t933000120138394\t933000120138397\t933000120138398\t933000120138400\t933000120138403\t933000120138556\t933000120138557\t933000120138558\t933000120138561\t933000120138569\t933000120138578\t933000120138632\t933000120138643\t933000120138645\t933000120138649\t933000120138650\t933000120138387\t933000120138389\t933000120138390\t933000120138391\t933000120138395\t933000120138399\t933000120138402\t933000120138405\t933000120138409\t933000120138414\t933000120138415\t933000120138427\t933000120138436\t933000120138437\t933000120138454\t933000120138455\t933000120138494\t933000120138496\t933000120138564\t933000120138568\t933000120138573\t933000120138575\t933000120138577\t933000120138579\t933000120138582\t933000120138583\t933000120138584\t933000120138585\t933000120138589\t933000120138592\t933000120138598\t933000120138599\t933000120138600\t933000120138601\t933000120138604\t933000120138605\t933000120138607\t933000120138609\t933000120138615\t933000120138616\t933000120138619\t933000120138627\t933000120138629\t933000120138631\t933000120138634\t933000120138636\t933000120138637\t933000120138638\t933000120138639\t933000120138640\t933000120138642\t933000120138644\t933000120138646\t933000120138648\t933000120138652\t933000120138654\t933000120138655\t933000120138656\t933000120138661\t933000120138662\t933000120138663\t933000120138665\t933000120138666\t933000120138667\t933000120138668\t933000120138669\t933000120138670\t933000120138672\t933000120138673\t933000120138674\t933000120138675\t933000120138679\t933000120138680\t933000120138686\t933000120138687\t933000120138689\t933000120138696\t933000120138698\t933000120138700\t933000120138708\t933000120138711\t933000120138717\t933000120138718\t933000120138724\t933000120138730\t933000120138734\t933000120138748\t933000120138750\t933000120138754\t933000120138406\t933000120138407\t933000120138410\t933000120138420\t933000120138424\t933000120138434\t933000120138447\t933000120138500\t933000120138504\t933000120138611\t933000120138630\t933000120138684\t933000120138722\t933000120138736\t933000120138744\t933000320045652\t933000320045653\t933000320045657\t933000320045659\t933000320045660\t933000320045661\t933000320045667\t933000320045670\t933000320045672\t933000320045674\t933000320045675\t933000320045735\t933000320045736\t933000320045740\t933000320045744\t933000320045777\t933000320045778\t933000320045781\t933000320045784\t933000320045785\t933000320045787\t933000320045791\t933000320045792\t933000320045794\t933000320045799\t933000320045988\t933000320045990\t933000320046078\t933000320046086\t933000320046088\t933000320046092\t933000320046093\t933000320046099\t933000320046127\t933000320046131\t933000320046135\t933000320

In [11]:
import time
plink = "/cm/shared/apps/plink/1.90-beta7/bin/plink"
ratgenes_master = "/projects/ps-palmer/hs_rats/round9_1/Heterogenous-stock_n14780_10182022_QC_Sex_Het_pass_n13526.vcf.gz"
ratgenes = "/scratch/capstone/Heterogenous-stock_n14780_10182022_QC_Sex_Het_pass_n13526.vcf.gz"

In [2]:
# Copy file over (if necessary)

tic = time.time()
! cp {ratgenes_master} {ratgenes}
toc = time.time()
print(tic-toc)

-175.34983205795288


In [13]:
# Create BED files
bfile_no_id = "/scratch/capstone/ratgenes_unpruned"

tic = time.time()
! {plink} --vcf {ratgenes} --make-bed --out {bfile_no_id}
toc = time.time()
print(tic-toc)

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /scratch/capstone/ratgenes_unpruned.log.
Options in effect:
  --make-bed
  --out /scratch/capstone/ratgenes_unpruned
  --vcf /scratch/capstone/Heterogenous-stock_n14780_10182022_QC_Sex_Het_pass_n13526.vcf.gz

95274 MB RAM detected; reserving 47637 MB for main workspace.
--vcf: /scratch/capstone/ratgenes_unpruned-temporary.bed +
/scratch/capstone/ratgenes_unpruned-temporary.bim +
/scratch/capstone/ratgenes_unpruned-temporary.fam written.
6533840 variants loaded from .bim file.
13526 people (0 males, 0 females, 13526 ambiguous) loaded from .fam.
Ambiguous sex IDs written to /scratch/capstone/ratgenes_unpruned.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 13526 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323

In [24]:
# Assign variant IDs
bfile = "/scratch/capstone/ratgenes_unpruned_id"
id_string = "@:#\$1,\$2"

tic = time.time()
! {plink} --bfile {bfile_no_id} --set-missing-var-ids {id_string} --make-bed --out {bfile} 
toc = time.time()
print(tic-toc)

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /scratch/capstone/ratgenes_unpruned_id.log.
Options in effect:
  --bfile /scratch/capstone/ratgenes_unpruned
  --make-bed
  --out /scratch/capstone/ratgenes_unpruned_id
  --set-missing-var-ids @:#$1,$2

95274 MB RAM detected; reserving 47637 MB for main workspace.
6533840 variants loaded from .bim file.
6533840 missing IDs set.
13526 people (0 males, 0 females, 13526 ambiguous) loaded from .fam.
Ambiguous sex IDs written to /scratch/capstone/ratgenes_unpruned_id.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 13526 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total gen

In [25]:
# Do LD pruning
window_size = 50
step_size = 5
r2_threshold = 0.05
prunefile = f"/scratch/capstone/ld{window_size}_{step_size}_{r2_threshold}"

tic = time.time()
! {plink} --bfile {bfile} --indep-pairwise {window_size} {step_size} {r2_threshold} --allow-no-sex --out {prunefile}
toc = time.time()
print(tic-toc)
! head {prunefile}.prune.in

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /scratch/capstone/ld50_5_0.05.log.
Options in effect:
  --allow-no-sex
  --bfile /scratch/capstone/ratgenes_unpruned_id
  --indep-pairwise 50 5 0.05
  --out /scratch/capstone/ld50_5_0.05

95274 MB RAM detected; reserving 47637 MB for main workspace.
6533840 variants loaded from .bim file.
13526 people (0 males, 0 females, 13526 ambiguous) loaded from .fam.
Ambiguous sex IDs written to /scratch/capstone/ld50_5_0.05.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 13526 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.993011.
6533840 variants and 13

In [26]:
# Extract pruned variants
final_file = "/scratch/capstone/ratgenes_pruned"

tic = time.time()
! {plink} --bfile {bfile} --extract {prunefile}.prune.in --make-bed --out {final_file}
toc = time.time()
print(tic-toc)

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /scratch/capstone/ratgenes_pruned.log.
Options in effect:
  --bfile /scratch/capstone/ratgenes_unpruned_id
  --extract /scratch/capstone/ld50_5_0.05.prune.in
  --make-bed
  --out /scratch/capstone/ratgenes_pruned

95274 MB RAM detected; reserving 47637 MB for main workspace.
6533840 variants loaded from .bim file.
13526 people (0 males, 0 females, 13526 ambiguous) loaded from .fam.
Ambiguous sex IDs written to /scratch/capstone/ratgenes_pruned.nosex .
--extract: 96654 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 13526 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979